### Cardiovascular data analysis
#### Randy J. Giedt

In [1]:
# Import Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mt
import seaborn as sns
import numpy as np
import datetime as dt
import math

#### Load datasets

Description of dataset codes: 

Subject ID (Subject ID) - an integer number identifying a particular patient.
This can be thought of as a substitute for a unique medical record
number. In the flat file data posted on PhysioNet, the number representing
the Subject ID is left padded with zeros to five digits and preceded by
the letter s. In the relational database, the Subject ID has no preceding
letter or leading zeros.

• Hospital admission ID (Hadm ID) - an integer number identifying a particular
admission to the hospital. Each patient may have many Hadm IDs
associated with his/her unique Subject ID.

• ICU stay ID (ICUstay ID) - an integer number identifying an ICU stay.
An ICU stay, refers to the period of time when the patient is cared for
continuosly in an Intensive Care Unit. Each patient may have one or more
ICU stays associated. An ICU stay is considered to be continuous if any
set of ICU events (such as bed transfers or changes in type of service)
belonging to one Subject ID which are fewer than 24 hours apart. Longer
breaks in the patient’s stay automatically cause a new ICUstay ID to be
assigned.

• Case ID (Case ID) - This is a five digit number preceded by the letter a (for
adults) or n (for neonates). This ID indicates a set of waveforms associated
with a given patient. For various reasons (described in section: 1.4.3
below), there may be multiple case IDs associated with a given patient.

In [2]:
# Comprises a description of what each ICD9 code stands for, with both a short title and a long title to search 
D_ICD_Diagnoses = pd.read_csv('/Users/jimmy/Documents/Large Data/Mimic/D_ICD_DIAGNOSES.csv')

# Comprises all ICD9 codes for each subject as identified by subject ID and HADM ID
Diagnoses_ICD = pd.read_csv('/Users/jimmy/Documents/Large Data/Mimic/DIAGNOSES_ICD.csv')

# List of all patients, their sex, DOB, DOD and associated information, by subject ID only
Patients = pd.read_csv('/Users/jimmy/Documents/Large Data/Mimic/PATIENTS.csv')

# Additional information for each patient including additional demographic info, diagnosis etc. 
Admissions = pd.read_csv('/Users/jimmy/Documents/Large Data/Mimic/ADMISSIONS.csv')

In [3]:
# Find Number of Unique Patients in over diagnosis database
unique_patients = Diagnoses_ICD.SUBJECT_ID.unique()
print(len(unique_patients))
# 46,520 unique patients in the data set
unique_visits = Diagnoses_ICD.HADM_ID.unique()
print(len(unique_visits))
# 58,976 unique visits in the data set

46520
58976


In [4]:
# Find the most common diagnoses in ICD9 data
unique_diagnoses = Diagnoses_ICD.ICD9_CODE.value_counts(); unique_diagnoses.head()

4019     20703
4280     13111
42731    12891
41401    12429
5849      9119
Name: ICD9_CODE, dtype: int64

#### 414.01, which is Coronary atherosclerosis of native coronary artery, has 12,429 diagnoses
#### 401 = Hypertension with 20,703 diagnoses
#### 428 = Heart Failure
#### 427 = Cardiac Dysrythmia 
#### 414.01 = Coronary atherosclerosis of native coronary artery

#### First, we will compile a bunch of metrics to get patient ages, stay lengths and several other metrics

In [5]:
# We need to calculate parameters for each patient including their age at death, age at discharge etc.

# Add Patient DOB in usable format and Merge with patient information
Patients_DOB = Patients['DOB']
Patients_DOB2 = []
for n in range(len(Patients_DOB)):
    prelim = Patients_DOB[n]
    Patients_DOB2.append(dt.datetime.strptime(prelim[0:10],'%Y-%m-%d').date())
Birth_Date_Series = pd.Series(Patients_DOB2)
Patients['DOB-2'] = Birth_Date_Series

# Add Data for Patient's Date of Death
Patients_DODeath = Patients['DOD']
Patients_DOD = []
for n in range(len(Patients_DODeath)):
    if pd.isnull(Patients_DODeath[n]) == True:  
        Patients_DOD.append(np.nan)
    else:
        prelim = Patients_DODeath[n]
        Patients_DOD.append(dt.datetime.strptime(prelim[0:10],'%Y-%m-%d').date())

# Add Data for Patient's Date of Admission
Patients_Admit = Admissions['ADMITTIME']
Patients_Admissions = []
for n in range(len(Patients_Admit)):
    if pd.isnull(Patients_Admit[n]) == True:  
        Patients_Admissions.append(np.nan)
    else:
        prelim = Patients_Admit[n]
        Patients_Admissions.append(dt.datetime.strptime(prelim[0:10],'%Y-%m-%d').date())

# Add Data for Patient's Date of Release
Patients_leave = Admissions['DISCHTIME']
Patients_Release = []
for n in range(len(Patients_leave)):
    if pd.isnull(Patients_leave[n]) == True:  
        Patients_Release.append(np.nan)
    else:
        prelim = Patients_leave[n]
        Patients_Release.append(dt.datetime.strptime(prelim[0:10],'%Y-%m-%d').date())        
       
# Add Data to Series       
Birth_Date_Series = pd.Series(Patients_DOB2)
Death_Series = pd.Series(Patients_DOD)

Admit_Series = pd.Series(Patients_Admissions)
Release_Series = pd.Series(Patients_Release)

# Find Age at Death
Age_Death = (Death_Series - Birth_Date_Series)
death_age = []

for entry in range(len(Age_Death)):
    if type(Age_Death[entry]) == float:
        death_age.append(float('NAN'))
    else:
        death_age.append(Age_Death[entry].days)

Age_Death = pd.Series(death_age)

# Find Total Admission Times
Admissions_Time = (Release_Series - Admit_Series)

Admit_Time = []

for entry in range(len(Admissions_Time)):
    if type(Admissions_Time[entry]) == float:
        Admit_Time.append(float('NAN'))
    else:
        Admit_Time.append(Admissions_Time[entry].days)

Admissions_Time = pd.Series(Admit_Time)
        
# Add to Pandas Dataframe
Patients['DOB-2'] = Birth_Date_Series
Patients['DOD-2'] = Death_Series
Patients['AOD'] = Age_Death

Admissions['Admit'] = Admit_Series
Admissions['Release'] = Release_Series
Admissions['Total Admission Time'] = Admissions_Time 

Admissions_culled = Admissions.drop_duplicates('SUBJECT_ID', keep = 'last')
Admissions_culled_Admits = Admissions_culled['ADMITTIME']

Admit_Ages = (Admit_Series - Birth_Date_Series)

# Combine admissions information and patient's information
Admissions_culled = Admissions.copy(deep = False)
Admissions_culled = Admissions_culled.drop_duplicates('SUBJECT_ID', keep = 'last')

Admissions_long = Admissions_culled.merge(Patients, on='SUBJECT_ID')
Admissions_long.head()
Admit_Series = pd.Series(Patients_DOD)
Admissions_culled_Admits = Admissions_culled['ADMITTIME']

Admit_times2 = pd.Series(Admissions_long['ADMITTIME'])
Atimes = []
for n in range(len(Admit_times2)):
    if pd.isnull(Admit_times2[n]) == True:  
        Atimes.append(np.nan)
    else:
        prelim = Admit_times2[n]
        Atimes.append(dt.datetime.strptime(prelim[0:10],'%Y-%m-%d').date())

DOB_Admit_times = pd.Series(Admissions_long['DOB'])
DOBAtimes = []
for n in range(len(DOB_Admit_times)):
    if pd.isnull(DOB_Admit_times[n]) == True:  
        DOBAtimes.append(np.nan)
    else:
        prelim = DOB_Admit_times[n]
        DOBAtimes.append(dt.datetime.strptime(prelim[0:10],'%Y-%m-%d').date())

Atimes = pd.Series(Atimes)
DOBAtimes = pd.Series(DOBAtimes)
        
admit_births = (Atimes - DOBAtimes)

Admit_Timet = []
for entry in range(len(admit_births)):
    if type(admit_births[entry]) == float:
        Admit_Timet.append(float('NAN'))
    else:
        Admit_Timet.append(admit_births[entry].days)

Admit_Timet = pd.Series(Admit_Timet)

# Find total number of visits for each subject ID
visit_count = pd.DataFrame(Admissions.SUBJECT_ID.value_counts())
visit_count = pd.DataFrame(Admissions.SUBJECT_ID.value_counts())
visit_count.reset_index(level=0, inplace=True)
visit_count.columns = ['SUBJECT_ID', 'ADMISSIONS']

Admissions_long = Admissions_long.merge(visit_count, on='SUBJECT_ID')

In [6]:
# Find causes of death for patients based on final admission reason
cdeath = []
subjectid = []
ldeath = []
ideath = []
edeath = []
tdeath = []
hamdiddeath = []
admitt = []

for entry in range(len(Admissions)):
    if Admissions['HOSPITAL_EXPIRE_FLAG'][entry] == 0:
        continue
    elif Admissions['HOSPITAL_EXPIRE_FLAG'][entry] == 1:
        subjectid.append(Admissions['SUBJECT_ID'][entry])
        cdeath.append(Admissions['DIAGNOSIS'][entry])
        ldeath.append(Admissions['ADMISSION_LOCATION'][entry])
        tdeath.append(Admissions['ADMISSION_TYPE'][entry])
        ideath.append(Admissions['INSURANCE'][entry])
        edeath.append(Admissions['ETHNICITY'][entry])
        hamdiddeath.append(Admissions['HADM_ID'][entry])
        admitt.append(Admissions['Total Admission Time'][entry])
        
cdeath = pd.Series(cdeath)
subjectid = pd.Series(subjectid)
ldeath = pd.Series(ldeath)
ideath = pd.Series(ideath)
edeath = pd.Series(edeath)
tdeath = pd.Series(tdeath)
hamdiddeath = pd.Series(hamdiddeath)
admitt = pd.Series(admitt)

Death = pd.DataFrame()
Death['SUBJECT_ID'] = subjectid
#Death['HADM_ID'] = hamdiddeath
Death['CAUSE'] = cdeath
#Death['INSURANCE'] = ideath
#Death['ETHNICITY'] = edeath
#Death['ADMISSION_TYPE'] = tdeath
#Death['ADMISSION_LOCATION'] = ldeath
#Death['TOTAL_ADMISSION_TIME'] = admitt
Death.head()

,SUBJECT_ID,CAUSE
0,31,STATUS EPILEPTICUS
1,56,HEAD BLEED
2,61,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA
3,67,SUBARACHNOID HEMORRHAGE
4,84,"GLIOBLASTOMA,NAUSEA"


In [7]:
heart_death_desc = Death[Death['CAUSE'].str.contains("HEART|MYOCARDIAL|CARDIAC", na=False)==True]
heart_attack = Death[Death['CAUSE'].str.contains("MYOCARDIAL", na=False)==True]
del heart_attack['CAUSE']
heart_attack['HEART_ATTACK_FLAG'] = 1

/Users/jimmy/anaconda3/envs/Genetics/lib/python2.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [8]:
# Combine admissions information and patients information for an improved demographic dataframe
demographics = pd.DataFrame()
demographics['SUBJECT_ID'] = Admissions_long['SUBJECT_ID']
demographics['GENDER'] = Admissions_long['GENDER']
demographics['DOB'] = Admissions_long['DOB-2']
demographics['DOD'] = Admissions_long['DOD-2']
demographics['DOA'] = Admissions_long['ADMITTIME']
demographics['ADMIT_AGE'] = Admit_Timet
demographics['ETHNICITY'] = Admissions_long['ETHNICITY']
demographics['MARITAL_STATUS'] = Admissions_long['MARITAL_STATUS']
demographics['LANGUAGE'] = Admissions_long['LANGUAGE']
demographics['RELIGION'] = Admissions_long['RELIGION']
demographics['INSURANCE'] = Admissions_long['INSURANCE']
demographics['ADMISSION_LOCATION'] = Admissions_long['ADMISSION_LOCATION']
demographics['#ADMISSIONS'] = Admissions_long['ADMISSIONS']
demographics.head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS
0,22,F,2131-05-07,NaN,2196-04-09 12:26:00,23714,WHITE,MARRIED,NaN,UNOBTAINABLE,Private,EMERGENCY ROOM ADMIT,1
1,23,M,2082-07-17,NaN,2157-10-18 19:34:00,27486,WHITE,MARRIED,ENGL,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,2
2,24,M,2100-05-31,NaN,2139-06-06 16:14:00,14250,WHITE,SINGLE,NaN,PROTESTANT QUAKER,Private,TRANSFER FROM HOSP/EXTRAM,1
3,25,M,2101-11-21,NaN,2160-11-02 02:06:00,21531,WHITE,MARRIED,NaN,UNOBTAINABLE,Private,EMERGENCY ROOM ADMIT,1
4,26,M,2054-05-04,2128-02-25,2126-05-06 15:16:00,26299,UNKNOWN/NOT SPECIFIED,SINGLE,NaN,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,1


In [9]:
# Merge Death and demographics on subject ID to add relevant data about patient deaths
demographics = pd.merge(Death, demographics, on= 'SUBJECT_ID', how = 'outer')

In [10]:
# Put "outside hospital" and create flag for patients who died outside of hospital and we don't have information on
death_cause = []
outside_death_cause = []
for entry in range(len(demographics)):
    if pd.isnull(demographics['DOD'][entry]) == False and pd.isnull(demographics['CAUSE'][entry]) == True:
        death_cause.append('Death Outside of Hospital')
        outside_death_cause.append(1)
    else: 
        death_cause.append(demographics['CAUSE'][entry])
        outside_death_cause.append(0)
        
death_cause = pd.Series(death_cause)
outside_death_flag = pd.Series(outside_death_cause)

demographics['CAUSE'] = death_cause
demographics['OUTSIDE_DEATH_FLAG'] = outside_death_flag
demographics.head()

# Add general death flag
death_flag = []
for entry in range(len(demographics)):
     if pd.isnull(demographics['DOD'][entry]) == False:
            death_flag.append(1)
     else: 
        death_flag.append(0)
        
death_flag = pd.Series(death_flag)
demographics['DEATH_FLAG'] = death_flag

# Add too old flag
old_flag = []
for entry in range(len(demographics)):
    if demographics['ADMIT_AGE'][entry] > 32850:
        old_flag.append(1)
    else:
        old_flag.append(0)

old_flag = pd.Series(old_flag)
demographics['OLD_FLAG'] = old_flag

#### To make ICD9 code searching easier I will simplify the listing of ICD9 codes to make them more searchable.

In [11]:
# Develop no letter list
list_values = Diagnoses_ICD['ICD9_CODE'].values.tolist()
no_letter_list = []
for i in range(len(list_values)):
    Value = str(list_values[i])
    Letter_stripped_value = Value.lstrip('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
    no_letter_list.append(Letter_stripped_value)

    # Develop no number list
list_values = Diagnoses_ICD['ICD9_CODE'].values.tolist()
no_number_list= []
for i in range(len(list_values)):
    Value = str(list_values[i])
    number_stripped_value = Value.lstrip('1234567890')
    if number_stripped_value:
        final_number = number_stripped_value[0]
        no_number_list.append(final_number)
    else:
        no_number_list.append('')      
        
# Correct three digit codes for V codes where decimal is only in first two spots
three_number_list = []
for i in range(len(list_values)):
    if no_number_list[i] == 'V':
        Value = no_letter_list[i] 
        three_digits = Value[0:2]
        three_number_list.append(float(three_digits))
    else:
        Value = no_letter_list[i]
        three_digits = Value[0:3]
        three_number_list.append(float(three_digits))

# Add three number list to diagnoses codes
three_number_series = pd.Series(three_number_list)
Diagnoses_ICD['Three Numbers'] = three_number_series # Pull only first three numbers (need to fix for letter codes)

In [12]:
# Fix ages for over 90 population
age = []
for entry in range(len(demographics)):
     if demographics['OLD_FLAG'][entry] == 0:
            age.append(demographics['ADMIT_AGE'][entry])
     else: 
        age.append(np.nan)
age = pd.Series(age)
demographics['ADMIT_AGE'] = age
demographics.head()

,SUBJECT_ID,CAUSE,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG
0,31,STATUS EPILEPTICUS,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394.0,WHITE,MARRIED,NaN,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,1,0,1,0
1,56,HEAD BLEED,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,Medicare,EMERGENCY ROOM ADMIT,1,0,1,1
2,61,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163.0,WHITE,MARRIED,NaN,CATHOLIC,Private,CLINIC REFERRAL/PREMATURE,2,0,1,0
3,67,SUBARACHNOID HEMORRHAGE,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842.0,WHITE,SINGLE,NaN,JEWISH,Medicare,EMERGENCY ROOM ADMIT,2,0,1,0
4,84,"GLIOBLASTOMA,NAUSEA",F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247.0,WHITE,MARRIED,NaN,OTHER,Private,EMERGENCY ROOM ADMIT,2,0,1,0


#### 2014 ICD-9-CM Diagnosis Code 414.01 : Coronary atherosclerosis of native coronary artery.

In [13]:
# Add myocardial infarction flag to data
demographics = pd.merge(demographics, heart_attack, on= 'SUBJECT_ID', how = 'outer')
demographics.head()

,SUBJECT_ID,CAUSE,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,HEART_ATTACK_FLAG
0,31,STATUS EPILEPTICUS,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394.0,WHITE,MARRIED,NaN,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,1,0,1,0,NaN
1,56,HEAD BLEED,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,Medicare,EMERGENCY ROOM ADMIT,1,0,1,1,NaN
2,61,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163.0,WHITE,MARRIED,NaN,CATHOLIC,Private,CLINIC REFERRAL/PREMATURE,2,0,1,0,NaN
3,67,SUBARACHNOID HEMORRHAGE,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842.0,WHITE,SINGLE,NaN,JEWISH,Medicare,EMERGENCY ROOM ADMIT,2,0,1,0,NaN
4,84,"GLIOBLASTOMA,NAUSEA",F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247.0,WHITE,MARRIED,NaN,OTHER,Private,EMERGENCY ROOM ADMIT,2,0,1,0,NaN


In [14]:
attack_flag = []

for entry in range(len(demographics)):
    if pd.isnull(demographics['HEART_ATTACK_FLAG'][entry]) == True:
        attack_flag.append(0)
    else:
        attack_flag.append(1)

attack_flag = pd.Series(attack_flag)
demographics['HEART_ATTACK_FLAG'] = attack_flag

In [15]:
# Pull Data for all Coronary artery disease patients
Cor_Diagnoses_ICD = Diagnoses_ICD.loc[Diagnoses_ICD['ICD9_CODE'] == '41401'] # All Coronary Artery disease patients
Athero_Diagnoses = pd.DataFrame(Cor_Diagnoses_ICD.SUBJECT_ID)
Athero_Diagnoses = Athero_Diagnoses.drop_duplicates()
Athero_Diagnoses['ATHERO_DIAGNOSIS_FLAG'] = 1


# Merge with demographics file:
demographics = pd.merge(demographics, Athero_Diagnoses, on= 'SUBJECT_ID', how = 'outer')

In [16]:
# Add Atherosclerosis diagnosis flags to demographic data
athero_flag = []
for entry in range(len(demographics)):
    if demographics['ATHERO_DIAGNOSIS_FLAG'][entry] == 1:
        athero_flag.append(1)
    else:
        athero_flag.append(0)
athero_flag = pd.Series(athero_flag)
demographics['ATHERO_DIAGNOSIS_FLAG'] = athero_flag

In [17]:
# Add heart cause of death flag to demographic data
demographics = pd.merge(demographics, heart_death_desc, on= 'SUBJECT_ID', how = 'outer')
demographics.head()

,SUBJECT_ID,CAUSE_x,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,HEART_ATTACK_FLAG,ATHERO_DIAGNOSIS_FLAG,CAUSE_y
0,31,STATUS EPILEPTICUS,M,2036-05-17,2108-08-30,2108-08-22 23:27:00,26394.0,WHITE,MARRIED,NaN,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,1,0,1,0,0,0,NaN
1,56,HEAD BLEED,F,1804-01-02,2104-01-08,2104-01-02 02:01:00,NaN,WHITE,NaN,NaN,NOT SPECIFIED,Medicare,EMERGENCY ROOM ADMIT,1,0,1,1,0,0,NaN
2,61,NON-HODGKINS LYMPHOMA;FEBRILE;NEUTROPENIA,M,2063-10-21,2119-02-03,2119-01-04 18:12:00,20163.0,WHITE,MARRIED,NaN,CATHOLIC,Private,CLINIC REFERRAL/PREMATURE,2,0,1,0,0,0,NaN
3,67,SUBARACHNOID HEMORRHAGE,M,2084-06-05,2157-12-02,2157-12-02 00:45:00,26842.0,WHITE,SINGLE,NaN,JEWISH,Medicare,EMERGENCY ROOM ADMIT,2,0,1,0,0,0,NaN
4,84,"GLIOBLASTOMA,NAUSEA",F,2151-10-21,2196-04-17,2196-04-14 04:02:00,16247.0,WHITE,MARRIED,NaN,OTHER,Private,EMERGENCY ROOM ADMIT,2,0,1,0,0,0,NaN


In [18]:
heart_death_flag = []

for entry in range(len(demographics)):
    if pd.isnull(demographics['CAUSE_y'][entry]) == True:
        heart_death_flag.append(0)
    else:
        heart_death_flag.append(1)

heart_death_flag = pd.Series(heart_death_flag)
demographics['HEART_DEATH_FLAG'] = athero_flag

del demographics['CAUSE_y']

demographics['CAUSE'] = demographics['CAUSE_x']
del demographics['CAUSE_x']
demographics['HEART_DEATH_FLAG'] = heart_death_flag

In [19]:
# Add Age in years to simplify interpretation
demographics['ADMIT_AGE'] = demographics['ADMIT_AGE']/365

In [20]:
# Test: Find all demographic information on patients who died from heart condition
demographics[demographics['HEART_DEATH_FLAG'] == 1].head()

,SUBJECT_ID,GENDER,DOB,DOD,DOA,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,HEART_ATTACK_FLAG,ATHERO_DIAGNOSIS_FLAG,HEART_DEATH_FLAG,CAUSE
8,106,M,2164-04-01,2192-08-15,2192-08-09 18:34:00,28.375342,UNKNOWN/NOT SPECIFIED,NaN,NaN,UNOBTAINABLE,Medicaid,EMERGENCY ROOM ADMIT,1,0,1,0,0,0,1,CARDIAC ARREST
19,305,F,2052-10-24,2129-09-07,2129-08-20 18:03:00,76.871233,WHITE,MARRIED,PORT,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,4,0,1,0,0,1,1,HEART FAILURE
31,491,M,2090-04-25,2170-04-05,2170-04-02 21:41:00,79.989041,WHITE,MARRIED,NaN,JEWISH,Medicare,EMERGENCY ROOM ADMIT,1,0,1,0,1,0,1,CHEST PAIN;RULE-OUT MYOCARDIAL INFARCTION
32,495,M,2106-09-23,2183-05-22,2183-05-13 15:08:00,76.687671,WHITE,SINGLE,NaN,EPISCOPALIAN,Medicare,CLINIC REFERRAL/PREMATURE,1,0,1,0,0,0,1,HEART FAILURE;KIDNEY RENAL FAILURE
37,391,M,2024-07-03,2108-10-05,2108-09-26 15:00:00,84.287671,UNKNOWN/NOT SPECIFIED,MARRIED,NaN,CATHOLIC,Medicare,TRANSFER FROM HOSP/EXTRAM,1,0,1,0,0,1,1,CORONARY ARTERY DISEASE\CARDIAC CATH


In [21]:
# Split List into those whose age we know, those still alive, and those too old to have age listed
# Combine those still alive with those who died at a known age
Patients_old = demographics[demographics['OLD_FLAG'] == 1]
Patients_young = demographics[demographics['OLD_FLAG'] == 0]
Patients_alive = demographics[demographics['DEATH_FLAG'] == 0]
Patients_dead = demographics[demographics['DEATH_FLAG'] == 1]
print('# of total patients is %d') % len(demographics)
print('# of old patients is %d' % len(Patients_old))
print('# of young patients is %d' % len(Patients_young))
print('# of living patients is %d' % len(Patients_alive))
print('# of dead patients is %d' % len(Patients_dead))

# of total patients is 46561
# of old patients is 1991
# of young patients is 44570
# of living patients is 30761
# of dead patients is 15800


In [22]:
# Fill na values with "uknown" outside of age
demographics = demographics.fillna({'ETHNICITY':'UKNOWN','MARITAL_STATUS':'UKNOWN', 'RELIGION': 'UKNOWN', 'LANGUAGE':'UKNOWN', 'INSURANCE':'UKNOWN', 'ADMISSION_LOCATION':'UKNOWN'})

#### To summarize to this point, we have created: 1. a new demographics sheet with information about all patients including who was diagnoses with coronary atherosclerosis, who lived and who died, who died of a heart related condition etc. With this information we can next analyze the patient populations to gain insight into how demographic factors may provide prediction value.

### Preliminary Data Analysis

In [23]:
demographics.groupby('GENDER').mean()

,SUBJECT_ID,ADMIT_AGE,#ADMISSIONS,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,HEART_ATTACK_FLAG,ATHERO_DIAGNOSIS_FLAG,HEART_DEATH_FLAG
GENDER,,,,,,,,,
F,34395.473517,51.872883,1.276251,0.222696,0.354990,0.063403,0.003136,0.179382,0.011515
M,34423.208359,51.317735,1.262427,0.206562,0.327126,0.026652,0.002944,0.272102,0.011204


#### Looking at our data set, the male population had a far higher likelihood of being diagnosed with coronoary atherosclerosis, but a similar (small chance) of experiencing a heart attack within the population found at this hopsital. 

In [24]:
demographics.groupby('INSURANCE').mean()

,SUBJECT_ID,ADMIT_AGE,#ADMISSIONS,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,HEART_ATTACK_FLAG,ATHERO_DIAGNOSIS_FLAG,HEART_DEATH_FLAG
INSURANCE,,,,,,,,,
Government,34431.435829,33.601264,1.157086,0.074198,0.133021,0.000668,0.000000,0.116310,0.002674
Medicaid,34405.221409,35.041305,1.297347,0.126944,0.212717,0.003888,0.000229,0.097210,0.005947
Medicare,38173.082174,73.243174,1.383479,0.354606,0.540848,0.091352,0.005402,0.341030,0.019265
Private,30335.024623,35.562743,1.151900,0.095627,0.168766,0.003019,0.001353,0.156169,0.004685
Self Pay,33697.480496,38.750768,1.046099,0.046099,0.203901,0.007092,0.001773,0.078014,0.008865


In [25]:
demographics.groupby('#ADMISSIONS').mean()

,SUBJECT_ID,ADMIT_AGE,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,HEART_ATTACK_FLAG,ATHERO_DIAGNOSIS_FLAG,HEART_DEATH_FLAG
#ADMISSIONS,,,,,,,,
1,34638.924685,49.433908,0.192443,0.304056,0.040941,0.003181,0.211528,0.010133
2,34187.033282,61.811679,0.304540,0.489996,0.050789,0.002501,0.307234,0.015198
3,34061.492560,65.186714,0.372768,0.577381,0.061756,0.002976,0.369792,0.026786
4,29136.781925,64.307946,0.337917,0.585462,0.053045,0.000000,0.408644,0.013752
5,25733.471545,63.771361,0.382114,0.626016,0.040650,0.000000,0.406504,0.016260
6,25840.283186,63.594674,0.336283,0.637168,0.053097,0.000000,0.424779,0.035398
7,28175.392157,60.421096,0.294118,0.549020,0.019608,0.000000,0.450980,0.019608
8,28306.451613,60.123023,0.387097,0.580645,0.000000,0.000000,0.451613,0.032258
9,20859.153846,62.096628,0.307692,0.692308,0.000000,0.000000,0.423077,0.000000


In [26]:
demographics[demographics['ATHERO_DIAGNOSIS_FLAG'] == 1].groupby('#ADMISSIONS').size() 

#ADMISSIONS
1     8246
2     1597
3      497
4      208
5      100
6       48
7       23
8       14
9       11
10      10
11       7
12       4
13       3
16       2
17       2
19       1
21       1
23       1
24       1
42       1
dtype: int64

In [27]:
demographics.groupby('ATHERO_DIAGNOSIS_FLAG').mean()

,SUBJECT_ID,ADMIT_AGE,#ADMISSIONS,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,HEART_ATTACK_FLAG,HEART_DEATH_FLAG
ATHERO_DIAGNOSIS_FLAG,,,,,,,,
0,33876.483177,46.190726,1.220657,0.196680,0.329058,0.039599,0.001453,0.008467
1,36186.034147,69.627211,1.427299,0.269927,0.373481,0.053262,0.008258,0.020878


In [28]:
demographics.groupby('MARITAL_STATUS').mean()

,SUBJECT_ID,ADMIT_AGE,#ADMISSIONS,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,HEART_ATTACK_FLAG,ATHERO_DIAGNOSIS_FLAG,HEART_DEATH_FLAG
MARITAL_STATUS,,,,,,,,,
DIVORCED,40006.459691,62.276258,1.376072,0.272727,0.409520,0.017153,0.000429,0.272727,0.008148
LIFE PARTNER,63181.076923,55.179136,1.461538,0.153846,0.153846,0.000000,0.000000,0.307692,0.000000
MARRIED,37915.761280,64.887383,1.306695,0.242935,0.383963,0.027449,0.003512,0.333009,0.013292
SEPARATED,36035.367188,58.376208,1.513021,0.218750,0.359375,0.010417,0.000000,0.208333,0.010417
SINGLE,39679.142068,49.963485,1.361358,0.193413,0.305002,0.024343,0.002250,0.159251,0.008489
UKNOWN,19791.282675,13.055875,1.036817,0.055737,0.123338,0.013397,0.001432,0.042340,0.005523
UNKNOWN (DEFAULT),32384.394137,65.562132,1.117264,0.335505,0.547231,0.068404,0.003257,0.257329,0.016287
WIDOWED,36820.030752,77.706194,1.332418,0.400879,0.593630,0.192019,0.006956,0.337543,0.021417


In [29]:
demographics.groupby('ETHNICITY').mean()

,SUBJECT_ID,ADMIT_AGE,#ADMISSIONS,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,HEART_ATTACK_FLAG,ATHERO_DIAGNOSIS_FLAG,HEART_DEATH_FLAG
ETHNICITY,,,,,,,,,
AMERICAN INDIAN/ALASKA NATIVE,22393.636364,22.282939,1.136364,0.113636,0.181818,0.000000,0.000000,0.090909,0.000000
AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE,48575.000000,52.306849,1.500000,0.500000,1.000000,0.000000,0.000000,0.000000,0.000000
ASIAN,23682.855050,26.569184,1.158057,0.104086,0.185042,0.020046,0.000771,0.084040,0.004626
ASIAN - ASIAN INDIAN,54308.413793,52.082094,1.568966,0.068966,0.120690,0.034483,0.000000,0.224138,0.000000
ASIAN - CAMBODIAN,61233.909091,54.512827,1.636364,0.181818,0.545455,0.000000,0.090909,0.363636,0.090909
ASIAN - CHINESE,49858.337662,52.959503,1.233766,0.116883,0.233766,0.030303,0.000000,0.164502,0.004329
ASIAN - FILIPINO,56435.933333,53.490411,1.666667,0.133333,0.266667,0.066667,0.000000,0.266667,0.066667
ASIAN - JAPANESE,58800.428571,56.310502,1.000000,0.000000,0.285714,0.142857,0.000000,0.142857,0.000000
ASIAN - KOREAN,56058.166667,53.418721,1.166667,0.083333,0.166667,0.000000,0.000000,0.166667,0.000000


In [30]:
demographics.groupby('ADMISSION_LOCATION').mean()

,SUBJECT_ID,ADMIT_AGE,#ADMISSIONS,OUTSIDE_DEATH_FLAG,DEATH_FLAG,OLD_FLAG,HEART_ATTACK_FLAG,ATHERO_DIAGNOSIS_FLAG,HEART_DEATH_FLAG
ADMISSION_LOCATION,,,,,,,,,
** INFO NOT AVAILABLE **,21137.763819,0.834994,1.000000,0.010050,0.025126,0.005025,0.000000,0.000000,0.005025
CLINIC REFERRAL/PREMATURE,50512.374061,50.014977,1.385879,0.145018,0.258888,0.049775,0.001903,0.215223,0.009915
EMERGENCY ROOM ADMIT,30047.632710,61.286446,1.329435,0.308199,0.496307,0.070848,0.002770,0.172781,0.013973
HMO REFERRAL/SICK,15715.180000,0.611178,1.000000,0.010000,0.040000,0.000000,0.000000,0.000000,0.000000
PHYS REFERRAL/NORMAL DELI,27891.320531,35.336562,1.152381,0.112724,0.147931,0.006635,0.000390,0.195082,0.002498
TRANSFER FROM HOSP/EXTRAM,34278.477194,64.160154,1.166236,0.275961,0.438468,0.029834,0.010184,0.467871,0.023379
TRANSFER FROM OTHER HEALT,36169.980000,64.818712,2.240000,0.280000,0.660000,0.060000,0.000000,0.260000,0.020000
TRANSFER FROM SKILLED NUR,28537.748718,74.313027,1.610256,0.548718,0.810256,0.235897,0.005128,0.241026,0.025641
TRSF WITHIN THIS FACILITY,66104.000000,49.872603,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [31]:
lang = demographics.groupby('LANGUAGE').mean()
language = pd.DataFrame()
language['English'] = lang.loc['ENGL']
lang = lang.drop('ENGL')
language['Others'] = lang.mean()
language

,English,Others
SUBJECT_ID,53097.312735,44496.716392
ADMIT_AGE,61.641422,68.752789
#ADMISSIONS,1.383941,1.408095
OUTSIDE_DEATH_FLAG,0.192131,0.230451
DEATH_FLAG,0.306997,0.429934
OLD_FLAG,0.049766,0.115610
HEART_ATTACK_FLAG,0.001745,0.004094
ATHERO_DIAGNOSIS_FLAG,0.266413,0.218231
HEART_DEATH_FLAG,0.009916,0.040907


In [160]:
# Export data set
demographics.to_csv('Demographics.csv')